In [ ]:
# Load EDA Pkgs
import pandas as pd
import numpy as np

In [ ]:
# Load Data Viz Pkgs
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ML Pkgs
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.metrics import accuracy_score,hamming_loss,classification_report

In [ ]:
### Split Dataset into Train and Text
from sklearn.model_selection import train_test_split
# Feature engineering
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
# Multi Label Pkgs
from skmultilearn.problem_transform import BinaryRelevance
from skmultilearn.problem_transform import ClassifierChain
from skmultilearn.problem_transform import LabelPowerset
from skmultilearn.adapt import MLkNN

In [ ]:
train_path = "/kaggle/input/code-injection-6labels-balance/code_injection_6labels_balance.csv"


In [ ]:
# Load Dataset
df = pd.read_csv(train_path)

In [ ]:
target_list = ["000 - Normal", '126 - Path Traversal',
               '242 - Code Injection', '274 - HTTP Verb Tampering',
               '66 - SQL Injection', '88 - OS Command Injection']

In [ ]:
df

In [ ]:
!pip install neattext

In [ ]:
import neattext as nt
import neattext.functions as nfx

In [ ]:
# Explore For Noise
df['text'].apply(lambda x:nt.TextFrame(x).noise_scan())

In [ ]:
# Explore For Noise
df['text'].apply(lambda x:nt.TextExtractor(x).extract_stopwords())

In [ ]:
# Explore For Noise
df['text'].apply(nfx.remove_stopwords)

In [ ]:
corpus = df['text'].apply(nfx.remove_stopwords)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
tfidf = TfidfVectorizer()

In [ ]:
# Build Features
Xfeatures = tfidf.fit_transform(corpus)

In [ ]:
len(tfidf.get_feature_names_out())

In [ ]:
print(Xfeatures.toarray())

In [ ]:
len(Xfeatures.toarray()[0])

In [ ]:
y = df[target_list]

In [ ]:
# Split Data 
X_train,X_test,y_train,y_test = train_test_split(Xfeatures,y,test_size=0.3,random_state=42)

In [ ]:
### Problem Transform
import skmultilearn

In [ ]:
def build_model(model,mlb_estimator,xtrain,ytrain,xtest,ytest):
    # Create an Instance
    clf = mlb_estimator(model)
    clf.fit(xtrain,ytrain)
    # Predict
    clf_predictions = clf.predict(xtest)
    # Check For Accuracy
    evaluate_model(ytest, clf_predictions)
    acc = accuracy_score(ytest,clf_predictions)
    ham = hamming_loss(ytest,clf_predictions)
    result = {"accuracy:":acc,"hamming_score":ham}
    return result

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report,accuracy_score, f1_score

In [ ]:
def hamming_score(y_true, y_pred, normalize=True, sample_weight=None):
    y_true1 = y_true.to_numpy()
    y_pred1 = y_pred.toarray()
    acc_list = []
    
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true1[i])[0] )
        set_pred = set( np.where(y_pred1[i])[0] )
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float(len(set_true.union(set_pred)))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

In [ ]:
clf = BinaryRelevance(MultinomialNB())
clf.fit(X_train,y_train)
# Predict
clf_predictions = clf.predict(X_test)
# Check For Accuracy

In [ ]:
hamming_score(y_test, clf_predictions)

In [ ]:
def evaluate_model(y_test, br_prediction):
    score = f1_score(y_test,br_prediction,average="micro")
    precision = precision_score(y_test,br_prediction,average="micro")
    recall = recall_score(y_test,br_prediction,average="micro")
    report = classification_report(y_test,br_prediction,digits=4)
    EMR = accuracy_score(y_test,br_prediction)
    acc = hamming_score(y_test,br_prediction)
    
    #classifaction_report_csv(report,precision,recall,score,0)
    print ('\n clasification report:\n', report)
    print ('F1 score:', score)
    print ('Recall:', recall)
    print ('Precision:', precision)
    print('EMR:', EMR)
    print ('Acc:', acc)
    print('Hamming Loss', hamming_loss(y_test,br_prediction))
 

In [ ]:
binary_rel_clf = build_model(MultinomialNB(),BinaryRelevance,X_train,y_train,X_test,y_test)

In [ ]:
clf_chain_model = build_model(MultinomialNB(),ClassifierChain,X_train,y_train,X_test,y_test)

In [ ]:
clf_labelP_model = build_model(MultinomialNB(),LabelPowerset,X_train,y_train,X_test,y_test)